In [23]:
import sys
sys.path.append("..")
sys.path.append("../experiments")

from experiments.constants import * 
from experiments.utils import *

from random import randint
from collections import OrderedDict
import json

In [24]:
SRC, TGT = SPANISH, BASQUE
LP_FOLDER = f"/netscratch/falcao/data/appraise-batches/{SRC}-{TGT}"

## Create batches

In [25]:
# configurations
_task_definition = OrderedDict({
    "TGT": 80,
    "CHK": 0,
    "REF": 10,
    "BAD": 10,
})

BATCHES_FILENAME = "batches.json"

In [26]:
TASK_DEFINITION = ":".join(map(str, _task_definition.values()))
SRC_FILE = f"{LP_FOLDER}/src.{SRC}"
REF_FILE = f"{LP_FOLDER}/ref.{TGT}"
SYSTEMS_FOLDER = f"{LP_FOLDER}/systems/"
BATCHES_PATH = f"{LP_FOLDER}/{BATCHES_FILENAME}"

In [31]:
# call the command from bash but using variables set in Python
! python manage.py CreateDirectAssessmentData \
    100 \
    $SRC.code3 \
    $TGT.code3 \
    $LP_FOLDER/src.$SRC \
    $LP_FOLDER/ref.$TGT \
    $SYSTEMS_FOLDER \
    $BATCHES_PATH \
    --task-definition $TASK_DEFINITION \
    --required-annotations 3 \
    --source-based \
    --create-ids \
    --all-batches

Using task definition: (80, 0, 10, 10)
Loaded 288 source segments
Loaded 288 reference segments
character_based = False
Loaded 288 system gtrans-fake.txt segments
Loaded 288 system mstrans-fake.txt segments
Creating /netscratch/falcao/data/appraise-batches/es-eu/batches.json.segments ... OK
Missing items is 64/80/576
Added 64 missing items rotating keys
Total number of batches is 8
0 10 10
chk_items: 0
ref_items: 10
bad_items: 10
chk_ids: []
ref_ids: [12, 46, 1, 5, 24, 13, 35, 26, 39, 15]
bad_ids: [6, 16, 41, 47, 0, 2, 22, 14, 18, 44]
empty_slots [53, 54, 57, 58, 59, 60, 61, 67, 69, 70, 71, 73, 75, 77, 78, 79, 80, 81, 82, 83, 84, 86, 87, 88, 90, 92, 93, 95, 98, 99]
len(batch_items): 100
len(batch_items) == None: 0
0 10 10
chk_items: 0
ref_items: 10
bad_items: 10
chk_ids: []
ref_ids: [10, 31, 11, 44, 48, 41, 43, 39, 26, 12]
bad_ids: [32, 24, 5, 4, 14, 28, 47, 15, 3, 49]
empty_slots [50, 51, 52, 56, 57, 58, 59, 63, 66, 67, 68, 69, 70, 71, 72, 73, 75, 77, 79, 80, 83, 84, 85, 86, 87, 88, 9

## Create campaign

In [28]:
# configuration
CAMPAIGN_NAME = "test1item"

In [29]:
manifest = {
    "CAMPAIGN_URL": "http://127.0.0.1:8000/dashboard/sso/",
    "CAMPAIGN_NAME": CAMPAIGN_NAME,
    "CAMPAIGN_KEY": CAMPAIGN_NAME,
    "CAMPAIGN_NO": randint(0,100),
    "REDUNDANCY": 1,

    "TASKS_TO_ANNOTATORS": [
        [ SRC.code3, TGT.code3, "uniform", 1, 1 ]
    ]
}


In [7]:
assert type(manifest["TASKS_TO_ANNOTATORS"]) == list and type(manifest["TASKS_TO_ANNOTATORS"][0]) == list

In [9]:
MANIFEST_PATH = f"{LP_FOLDER}/manifest.json"

with open(MANIFEST_PATH, mode="w+") as f:
    json.dump(manifest, f, indent=4)

In [10]:
! python manage.py StartNewCampaign \
    $MANIFEST_PATH \
    --batches-json $BATCHES_PATH

JSON manifest path: '/netscratch/falcao/data/appraise-batches/es-eu/manifest.json'
CSV output path: None
Excel output path: None
No task type found in the manifest file, assuming it is "Direct". If this is incorrect, define "TASK_TYPE" in the manifest file.
### Running InitCampaign
All languages: [('spa', 'eus')]
Identified superuser: falcao
Processed Market/Metadata instances
### Creating a new campaign
- '/netscratch/falcao/data/appraise-batches/es-eu/batches.json'
Batch: /netscratch/falcao/data/appraise-batches/es-eu/batches.json
  Market: spa_eus_test1item
  Metadata: spa->eus/test1item["1.0"]
Uploaded file name: Batches/batches_H2j8u8J.json
Campaign name: test1item
### Running validatecampaigndata
Campaign name: test1item
Batch name: Batches/batches_H2j8u8J.json
Validated 1 batches
### Running ProcessCampaignData
Batches/batches_H2j8u8J.json 1
15 gtrans-fake.txt
195 b'[gtrans] Goizeko lehen orduan anaiak agertu ziren duin-itxurak eginez, tronpeta erlastuen zarataz inguraturik, eta

In [22]:
from math import ceil

n_sents = 388
n_systems = 2

n_tgt, n_chk, n_ref, n_bad = _task_definition.values()
n_batches = ceil(n_sents * n_systems / n_tgt)

n_control_total = n_batches * (n_chk + n_ref + n_bad)

(n_sents * n_systems) + n_control_total

100